# L2c: Eigendecomposition using QR-decomposition
In this lecture, we continue our discussion of eigendecomposition where we introduce a technique to compute all eigenvalues (and the associated eigenvectors) for a square matrix. 

> __Learning Objectives:__
> 
> By the end of this lecture, you should be able to:
>
> * __Apply QR iteration to compute all eigenvalues and eigenvectors:__ Implement the QR iteration algorithm to compute the full eigendecomposition of square matrices, and understand convergence properties and stopping criteria.
> * __Understand QR decomposition and its role in eigencomputation:__ Decompose matrices into orthogonal and upper triangular factors, and recognize how this factorization enables iterative eigenvalue methods.
> * __Compare classical and modified Gram-Schmidt orthogonalization:__ Implement both algorithms to compute QR decompositions, understand their numerical differences, and explain why modified Gram-Schmidt produces more orthogonal vectors in finite precision.


Let's get started!
___

## Examples
Today, we will use the following examples to illustrate key concepts:

> [▶ Let's use QR-iteration to analyze stock prices](CHEME-5820-L2c-Example-QR-Iteration-Spring-2026.ipynb). In this example, we implement the QR-iteration method to compute the eigendecomposition of the covariance matrix generated from the daily log growth rates of stock prices.
___

## Concept review: Eigendecomposition
Suppose we have a real square matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$ which could be a measurement dataset, e.g., the columns of $\mathbf{A}$ represent feature 
vectors $\mathbf{x}_{1},\dots,\mathbf{x}_{m}$ or an adjacency array in a graph with $m$ nodes, etc. Eigenvalue-eigenvector problems involve finding a set of scalar values $\left\{\lambda_{1},\dots,\lambda_{m}\right\}$ called 
[eigenvalues](https://mathworld.wolfram.com/Eigenvalue.html) and a set of linearly independent vectors 
$\left\{\mathbf{v}_{1},\dots,\mathbf{v}_{m}\right\}$ called [eigenvectors](https://mathworld.wolfram.com/Eigenvector.html) such that:
$$
\begin{align*}
\mathbf{A}\cdot\mathbf{v}_{j} &= \lambda_{j}\cdot\mathbf{v}_{j}\qquad{j=1,2,\dots,m}
\end{align*}
$$
where $\mathbf{v}\in\mathbb{C}^{m}$ and $\lambda\in\mathbb{C}$. We can put the eigenvalues and eigenvectors together in matrix-vector form, which gives us an interesting matrix decomposition:
$$
\mathbf{A} = \mathbf{V}\cdot\text{diag}(\lambda)\cdot\mathbf{V}^{-1}
$$
where $\mathbf{V}$ denotes the matrix of eigenvectors, where the eigenvectors form the columns of the matrix $\mathbf{V}$, $\text{diag}(\lambda)$ denotes a diagonal matrix with the eigenvalues along the main diagonal, and $\mathbf{V}^{-1}$ denotes the inverse of the eigenvalue matrix.

### Symmetric real matrices
The eigendecomposition of a symmetric real matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$ has some special properties. 
First, all the eigenvalues $\left\{\lambda_{1},\lambda_{2},\dots,\lambda_{m}\right\}$ of the matrix $\mathbf{A}$ are real-valued.
Next, the eigenvectors $\left\{\mathbf{v}_{1},\mathbf{v}_{2},\dots,\mathbf{v}_{m}\right\}$ of the matrix $\mathbf{A}$ are orthogonal, i.e., $\left<\mathbf{v}_{i},\mathbf{v}_{j}\right> = 0$ for $i\neq{j}$. Finally, the (normalized) eigenvectors $\mathbf{v}_{j}/\lVert\mathbf{v}_{j}\rVert_{2}$ of a symmetric real-valued matrix 
form an orthonormal basis for the space spanned by the matrix $\mathbf{A}$ such that:
$$
\begin{align*}
\left<\hat{\mathbf{v}}_{i},\hat{\mathbf{v}}_{j}\right> &= \delta_{ij}\qquad\text{for}\quad{i,j\in\mathbf{A}}
\end{align*}
$$
where $\delta_{ij}$ is the Kronecker delta ($\delta_{ij} = 1$ if $i=j$, and $\delta_{ij}=0$ if $i\neq j$). 

> __Interpretation of eigenvalues and eigenvectors:__
> * Eigenvectors represent fundamental directions of the matrix $\mathbf{A}$. For a linear transformation, eigenvectors are the only vectors that do not change direction—they are scaled by a corresponding eigenvalue.
> * Eigenvalues are scale factors indicating how much the corresponding eigenvector is stretched or compressed during the transformation.
> * The eigendecomposition diagonalizes a matrix: $\text{diag}(\lambda) = \mathbf{V}^{-1}\cdot\mathbf{A}\cdot\mathbf{V}$. Eigenvalues also classify a matrix as positive or negative (semi)definite.
> * For symmetric matrices with positive entries, all eigenvalues are real-valued and eigenvectors are orthogonal.

___

## QR Iteration
[QR iteration](https://en.wikipedia.org/wiki/QR_algorithm) is a technique used for computing the eigenvalues and eigenvectors of square matrices $\mathbf{A}$. The algorithm relies upon the [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition) of the matrix $\mathbf{A}$.

> __QR Decomposition__
>
> The __QR decomposition__ of a (rectangular) matrix $\mathbf{A}\in\mathbb{R}^{n\times{m}}$ is a product of an orthogonal matrix $\mathbf{Q}\in\mathbb{R}^{n\times{n}}$ and an upper triangular matrix $\mathbf{R}\in\mathbb{R}^{n\times{m}}$:
> $$
\begin{align*}
\mathbf{A} &= \mathbf{Q}\mathbf{R}
\end{align*}
$$
> where $\mathbf{Q}^{\top}\mathbf{Q} = \mathbf{I}_{n}$. This factorization relies on an orthogonalization method, e.g., the classical or modified Gram-Schmidt which generates a set of mutually orthogonal vectors $\mathbf{q}_{1},\mathbf{q}_{2},\dots, \mathbf{q}_{n}$ starting from a set of linearly independent vectors $\mathbf{x}_{1},\mathbf{x}_{2},\dots,\mathbf{x}_{n}$. 

The core of the QR iteration algorithm involves iteratively decomposing a given matrix $\mathbf{A}$ into its $\mathbf{Q}$ and $\mathbf{R}$ factors and then reformulating the matrix for subsequent iterations. Under certain conditions, [QR iteration](https://en.wikipedia.org/wiki/QR_algorithm) will converge to a triangular matrix with the eigenvalues of the original matrix $\mathbf{A}$ listed on the diagonal.

Let's explore a pseudo-code implementation of the [QR iteration method](https://en.wikipedia.org/wiki/QR_algorithm) to estimate all eigenvalues and eigenvectors of a square matrix $\mathbf{A}$.

__Initialization__: Given a real-valued square matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$, we set $\mathbf{A}_{1}\gets\mathbf{A}$, specify $\epsilon>0$ as the convergence tolerance, `maxiter` as the maximum number of iterations, an iteration counter $k\gets{1}$, and a boolean variable $\texttt{converged}\gets{\texttt{false}}$ to indicate whether the algorithm has converged.

> __Parameter selection rules of thumb__:  
> * __Convergence tolerance__ $\epsilon$: A good starting point is $\epsilon = 10^{-6}$ to $10^{-8}$ for most applications. For higher precision, use $\epsilon = 10^{-10}$ to $10^{-12}$.  
> * __Maximum iterations__ `maxiter`: Use `maxiter` in the range of 100-300 for typical matrices. For clustered eigenvalues or ill-conditioned matrices, increase to 500-1000.  
> * __Eigenvalue separation affects convergence rate__: When eigenvalues are well-separated ($|\lambda_1| > |\lambda_2| > \cdots > |\lambda_m|$), the algorithm converges rapidly in $O(m)$ iterations. Clustered eigenvalues slow convergence significantly, potentially requiring hundreds or thousands of iterations.

While not $\texttt{converged}$ __do__:
- Compute the QR decomposition of the current matrix: $\mathbf{A}_{k} = \mathbf{Q}_{k}\mathbf{R}_{k}$.
- Form the next iteration matrix by reversing the factors: $\mathbf{A}_{k+1}\gets\mathbf{R}_{k}\mathbf{Q}_{k}$ (equivalent to $\mathbf{A}_{k+1} = \mathbf{Q}^{\top}_{k}\mathbf{A}_{k}\mathbf{Q}_{k}$, a similarity transformation).
- Check for convergence:
   1. If $\lVert \mathbf{A}_{k+1} - \mathbf{A}_{k} \rVert_{1}\leq\epsilon$ (where $\lVert\star\rVert_{1}$ is the matrix 1-norm), then set $\texttt{converged}\gets{\texttt{true}}$ and return the eigenvalues from the diagonal of $\mathbf{A}_{k+1}$.
   2. If $k\geq\texttt{maxiter}$, then set $\texttt{converged}\gets{\texttt{true}}$ to terminate the algorithm, return the diagonal elements of $\mathbf{A}_{k+1}$ as eigenvalue estimates (optionally flagging that convergence tolerance was not met).
   3. Increment $k\gets{k+1}$.

__Output__: Return the eigenvalues $\left\{\lambda_{1},\lambda_{2},\dots,\lambda_{m}\right\}$ extracted from the diagonal of the converged matrix $\mathbf{A}_{\star}$.

__Computing eigenvectors__: Once we have converged to matrix $\mathbf{A}_{\star}$ with eigenvalues on the diagonal, we compute the eigenvectors by solving the homogeneous system of linear algebraic equations for each eigenvalue $\lambda_{j}$:
$$
\left(\mathbf{A} - \lambda_{j}\mathbf{I} \right)\cdot\mathbf{v}_{j} = \mathbf{0}
$$
where $\mathbf{I}$ is the identity matrix. This system has a non-trivial solution since $\lambda_{j}$ is an eigenvalue of $\mathbf{A}$.

> __Example__
>
> [▶ Let's use QR-iteration to analyze stock prices](CHEME-5820-L2c-Example-QR-Iteration-Spring-2026.ipynb). In this example, we implement the QR-iteration method to compute the eigendecomposition of the covariance matrix generated from the daily log growth rates of stock prices.

Ok, but where does the QR decomposition come from? To answer this question, we need to review the [Gram-Schmidt orthogonalization procedure](CHEME-5820-L2c-Advanced-GramSchmidt-Algorithm-Spring-2026.ipynb).

___

## Summary
QR iteration computes all eigenvalues and eigenvectors of square matrices by iteratively applying QR decomposition and reforming the matrix to approach upper triangular form.

> __Key Takeaways:__
>
> * **QR iteration converges to upper triangular form with eigenvalues on the diagonal:** The algorithm repeatedly decomposes $\mathbf{A}_{k} = \mathbf{Q}_{k}\mathbf{R}_{k}$ and reforms $\mathbf{A}_{k+1} = \mathbf{R}_{k}\mathbf{Q}_{k}$, which is similar to $\mathbf{A}_{k}$. Under convergence, the diagonal elements of $\mathbf{A}_{\star}$ provide the eigenvalues.
> * **QR decomposition relies on orthogonalization methods:** Classical and Modified Gram-Schmidt both produce $\mathbf{Q}$ and $\mathbf{R}$ factors, but differ in computational order and numerical stability. Modified Gram-Schmidt updates remaining columns immediately, reducing roundoff error accumulation.
> * **QR iteration computes all eigenvalues simultaneously:** Unlike power iteration (which finds only the dominant eigenpair), QR iteration provides a complete eigendecomposition, making it suitable for general eigenvalue problems without prior knowledge of the spectrum.


QR iteration is a fundamental algorithm for computing eigendecompositions and forms the basis for production eigenvalue solvers in scientific computing.
___